In [ ]:

import numpy as np
import sklearn as sk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import torch.optim as optim
from sklearn.svm import LinearSVC
import pickle
import helpers
import torch as torch
from sklearn.linear_model import LogisticRegression
import os
%load_ext autoreload
%autoreload 2

In [ ]:
"""Paths"""
POS_TWEET_PATH = 'Tweet_Large_Files/twitter-datasets/train_pos.txt'
NEG_TWEET_PATH = "Tweet_Large_Files/twitter-datasets/train_neg.txt"
VOCAB_PATH = 'vocab.pkl'
"""change path for different glove embedding dim"""
GLOVE_EMB_PATH = "imported_embeddings/glove.twitter.27B.200d.npy"

BELOW decide which vocab to use

In [ ]:
with open(VOCAB_PATH, 'rb') as file:
        vocab = pickle.load(file)

BElOW you decide which wordvector genereation to use

In [ ]:
"""BOW"""
old_vectorizer = False
fs = []
xlen = []
vectorizer = CountVectorizer(stop_words="english")
for fn in [POS_TWEET_PATH, NEG_TWEET_PATH]:
        with open(fn, errors= "ignore") as file:
                try: 
                        fs += file.readlines()
                        xlen.append(len(fs))
                except:
                        print(file.readlines())
Ypos = np.ones(xlen[0])
Yneg = np.ones(xlen[1]-xlen[0])*-1
Y = np.concatenate((Ypos, Yneg))              
X = vectorizer.fit_transform(fs)

In [ ]:
"""GloVe emdeding"""
"""makes dir for saving embedded tweets"""
path = 'Embedded_tweets/'
pathExist = os.path.exists(path)
if not pathExist:
    os.makedirs(path)
Xneg = helpers.build_tweet_vector(GLOVE_EMB_PATH,VOCAB_PATH,NEG_TWEET_PATH,"Embedded_tweets/embeddings_neg_imp_dim200.npy", TFID_weighting = False)
print("pos")
Xpos = helpers.build_tweet_vector(GLOVE_EMB_PATH,VOCAB_PATH,POS_TWEET_PATH,"Embedded_tweets/embeddings_pos_imp_dim200.npy", TFID_weighting = False)
Ypos = np.ones(Xpos.shape[0])
Yneg = np.ones(Xneg.shape[0])*-1
X = np.concatenate((Xneg,Xpos))
Y = np.concatenate((Yneg, Ypos))

In [ ]:
"""TF-ID vectorizer"""
old_vectorizer = False
fs = []
xlen = []
vectorizer = TfidfVectorizer(stop_words="english")
for fn in [POS_TWEET_PATH, NEG_TWEET_PATH]:
        with open(fn, errors= "ignore") as file:
                try: 
                        fs += file.readlines()
                        xlen.append(len(fs))
                except:
                        print(file.readlines())
Ypos = np.ones(xlen[0])
Yneg = np.ones(xlen[1]-xlen[0])*-1
Y = np.concatenate((Ypos, Yneg))              
X = vectorizer.fit_transform(fs)

Split data in train and test


In [ ]:
"""Splits training data"""
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=1)

Decide which supervised ML method to use:

In [ ]:
"""Multinomial naive bayes"""
MNB = MultinomialNB()
X_train = np.abs(X_train)
X_test = np.abs(X_test)
y_pred = MNB.fit(X_train, Y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"% (X_test.shape[0], (Y_test != y_pred).sum()))

In [ ]:
"""SVM"""
SVM = LinearSVC(random_state=0, tol=1e-5, max_iter=5000)
SVM.fit(X_train,Y_train)
y_pred  = SVM.predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"% (X_test.shape[0], (Y_test != y_pred).sum()))

In [ ]:
"""Logistic regression"""
LogReg = LogisticRegression(random_state=0,max_iter=1000).fit(X_train, Y_train)
y_pred = LogReg.predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"% (X_test.shape[0], (Y_test != y_pred).sum()))

Below Neural net,
Some processing of the data need to be done before it can be trained, can only be used with Glove

In [ ]:
"""Convert matrices to torch tensor"""
X_t = torch.empty((X_train.shape[0],1,X_train.shape[1]))

Y_t = torch.zeros((len(Y_train)))
for i,y in enumerate(Y_train):
    X_t[i,0,:] = torch.from_numpy(X_train[i,:])
    if(y == -1):
        Y_t[i] = 0
    else:
        Y_t[i] = 1
test = torch.tensor(np.array([[1, 2, 3], [4, 5, 6]]))
print(Y_t.type)
print(test.type)
X_t_test = torch.empty((X_test.shape[0],1,X_test.shape[1]))
for i,y in enumerate(Y_test):
    X_t_test[i,0,:] = torch.from_numpy(X_test[i,:])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 100, 2)
        self.conv2 = nn.Conv1d(100, 100, 3)
        self.conv3 = nn.Conv1d(100, 100, 4)
        self.fc1 = nn.Linear(2*3*4*100-200, 128)
        self.fc2 = nn.Linear(128, 128)
        self.pool = nn.AvgPool1d(2, 2)
        self.fc3 = nn.Linear(128, 2)
        
    def forward(self, x):
        x = self.pool(self.conv1(x))
        x = self.pool(self.conv2(x))
        x = self.pool(self.conv3(x))
        x = torch.flatten(x, 1)
        #print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net = net.float()
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [ ]:
"""Import pre existing network if you dont want to train new one, get file from google drive"""
net.load_state_dict(torch.load("neural_nets/layers2_nodes128_3cnn_filters100_size4_3_2.pth"))
prediction =   net(X_t_test.float())
_, y_pred = torch.max(prediction, 1)
y_pred = y_pred.detach().numpy()
y_pred[y_pred == 0] = -1
print("Number of mislabeled points out of a total %d points : %d"% (X_test.shape[0], (Y_test != y_pred).sum()))


In [ ]:
batch_size = 1000
for epoch in range(10):  # loop over the dataset multiple times
    permutation = torch.randperm(X_t.size()[0])
    running_loss = 0
    for i in range(0,X_t.size()[0], batch_size):
        optimizer.zero_grad()
        """gets batch indices"""
        indices = permutation[i:i+batch_size]
        """forward and backward"""
        outputs = net(X_t[indices].float())
        label = Y_t[indices].long()
        loss = criterion(outputs, label)
        loss.backward()
        """optimizing"""
        optimizer.step()
    prediction =   net(X_t_test.float())
    _, y_pred = torch.max(prediction, 1)
    y_pred = y_pred.detach().numpy()
    y_pred[y_pred == 0] = -1
    print("Number of mislabeled points out of a total %d points : %d"% (X_test.shape[0], (Y_test != y_pred).sum()))

In [ ]:
helpers.glove()